In [1]:
import os
if not os.path.exists("/scratch/sarthak"):
    os.makedirs("/scratch/sarthak")

os.environ["HF_HOME"] = "/scratch/sarthak/"

In [2]:
import torch
import torch.nn.functional as F
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModelForSeq2SeqLM
from typing import List, Tuple, Optional
import time
import numpy as np
# %pip install tqdm
# %pip install ipywidgets

In [3]:
from typing import Tuple
import torch
import datasets
from transformers import T5Tokenizer

# # WMT16 EN-DE dataset
# def preprocess_function_ende(examples, tokenizer, args, train_args, prefix="translate English to German: "):
#     if train_args.debug:
#         all_text = examples['translation'][:2]
#     else:
#         all_text = examples['translation']
        
#     inputs = []
#     targets = []
#     for excerpt in all_text:
#         en_text = prefix + excerpt['en']
#         de_text = excerpt['de']

#         inputs.append(en_text)
#         targets.append(de_text)
            
#     padding = 'max_length'
#     model_inputs = tokenizer(
#         inputs,
#         max_length=args.source_max_length,
#         padding=padding,
#         truncation=True,
#         return_tensors="pt",
#     )
#     # Tokenize targets with the `text_target` keyword argument
#     labels = tokenizer(
#         text_target=targets,
#         max_length=args.train_target_max_length,
#         padding=padding,
#         truncation=True,
    #     return_tensors="pt",
    # )

    # if padding == "max_length":
    #             labels["input_ids"] = [
    #                 [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    #             ]

    # model_inputs["labels"] = labels["input_ids"]
    # model_inputs["decoder_attention_mask"] = labels["attention_mask"]
    # return model_inputs

en_gr_dataset = datasets.load_dataset('wmt16', 'de-en')

# Base Speculative

In [4]:
class SpeculativeDecoder:
    def __init__(
        self,
        target_model_name = "google-t5/t5-3b",
        draft_model_name = "google-t5/t5-small",
        device = "cuda" if torch.cuda.is_available() else "cpu",
        gamma = 4,
        temperature = 0.5
    ):
        self.device = device
        self.gamma = gamma

        self.temperature = temperature

        self.tokenizer = T5Tokenizer.from_pretrained(target_model_name)
        
        # self.target_model = T5ForConditionalGeneration.from_pretrained(target_model_name).to(device)
        self.target_model = AutoModelForSeq2SeqLM.from_pretrained(target_model_name, device_map='auto')

        self.draft_model = T5ForConditionalGeneration.from_pretrained(draft_model_name).to(device)

        self.target_model.eval()
        self.draft_model.eval()

    def get_draft_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        gamma: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get draft logits for gamma tokens"""
        draft_tokens = []
        draft_probs = []
        current_decoder_ids = decoder_input_ids

        # Generate gamma tokens from the draft model
        for _ in range(gamma):
            with torch.no_grad():
                outputs = self.draft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=current_decoder_ids,
                    return_dict=True
                )
                logits = outputs.logits[:, -1, :]  # Get logits for last position
                probs = F.softmax(logits, dim=-1)

                # Sample token
                token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
                prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)

                draft_tokens.append(token_id.item())
                draft_probs.append(prob.item())

                # Update decoder inputs for next iteration
                current_decoder_ids = torch.cat(
                    [current_decoder_ids, token_id.view(1, 1)],
                    dim=1
                )

                if token_id.item() == self.tokenizer.eos_token_id:
                    break

        return draft_tokens, draft_probs#, current_decoder_ids, outputs.logits

    # def get_target_probs(
    #     self,
    #     input_ids: torch.Tensor,
    #     attention_mask: torch.Tensor,
    #     decoder_input_ids: torch.Tensor,
    #     draft_tokens: torch.Tensor
    # ) -> torch.Tensor:
    #     """Get target probabilities for the draft tokens in parallel."""
    #     with torch.no_grad():
    #         # Add draft tokens to decoder input
    #         # full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)

    #         full_decoder_ids = [decoder_input_ids]
    #         for i in range(len(draft_tokens)):
    #             x = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)[:, :i+1]], dim=1)
    #             full_decoder_ids.append(x)

    #         maxlen = max([x.shape[1] for x in full_decoder_ids])

    #         padded_decoder_ids = torch.stack([torch.tensor(
    #             F.pad(x, (0, maxlen - x.shape[1]), value=self.tokenizer.pad_token_id)[0]
    #         , device=self.device) for x in full_decoder_ids])

    #         batch_size = padded_decoder_ids.shape[0]
    #         input_ids_batched = input_ids.repeat(batch_size, 1)
    #         attention_mask_batched = attention_mask.repeat(batch_size, 1)

    #         # make it a triangular attention mask


    #         # print(input_ids_batched.shape, attention_mask_batched.shape, padded_decoder_ids.shape)

    #         # outputs = self.target_model(
    #         #     input_ids=input_ids,
    #         #     attention_mask=attention_mask,
    #         #     decoder_input_ids=full_decoder_ids,
    #         #     return_dict=True
    #         # )
    #         outputs = self.target_model(
    #             input_ids=input_ids_batched,
    #             attention_mask=attention_mask_batched,
    #             decoder_input_ids=padded_decoder_ids,
    #             # decoder_attention_mask=torch.triu(
    #             #     torch.zeros((padded_decoder_ids.shape[0], padded_decoder_ids.shape[1]), device=self.device)
    #             # ),
    #             return_dict=True
    #         )
    #         # print("passes target model")

    #         batched_logits = outputs.logits
    #         # print(batched_logits.shape)
    #         # 5, 5, 32128
    #         logits = torch.zeros((batched_logits.shape[1], batched_logits.shape[2]), device=self.device)
    #         for i in range(len(draft_tokens)):
    #             logits[i] = batched_logits[i, i, :]

    #         # print(logits.shape, batched_logits[-1].shape)

    #         target_probs = F.softmax(logits, dim=-1)

    #         # # Get probabilities for positions before each draft token
    #         # logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
    #         # target_probs = F.softmax(logits, dim=-1)

    #         # print(batched_logits[-1].unsqueeze(0).shape)

    #         return target_probs.squeeze(0), logits.unsqueeze(0)

    def get_target_probs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        draft_tokens: torch.Tensor
    ) -> torch.Tensor:
        """Get target probabilities for the draft tokens in parallel."""
        with torch.no_grad():
            # Add draft tokens to decoder input
            full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)

            # print(input_ids_batched.shape, attention_mask_batched.shape, padded_decoder_ids.shape)

            outputs = self.target_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=full_decoder_ids,
                # decoder_attention_mask=torch.triu(
                #     torch.zeros((full_decoder_ids.shape[0], full_decoder_ids.shape[1]), device=self.device)
                # ),
                return_dict=True
            )

            # Get probabilities for positions before each draft token
            logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
            target_probs = F.softmax(logits, dim=-1)

            return target_probs.squeeze(0), outputs.logits

    # def verify_tokens(
    #     self,
    #     target_probs: torch.Tensor,
    #     draft_tokens: torch.Tensor,
    #     draft_probs: torch.Tensor,
    # ) -> int:
    #     """Determine number of accepted tokens"""
    #     # Get target probabilities for the draft tokens
    #     target_probs_draft_tokens = target_probs.gather(
    #         -1,
    #         draft_tokens.unsqueeze(-1)
    #     ).squeeze(-1)

    #     # Calculate acceptance ratios
    #     acceptance_ratios = target_probs_draft_tokens / draft_probs

    #     # Sample uniform random numbers
    #     random_nums = torch.rand_like(acceptance_ratios)

    #     # Find number of accepted tokens
    #     # Accept if random number < min(1, target_prob / draft_prob)
    #     accepts = random_nums < torch.minimum(
    #         torch.ones_like(acceptance_ratios),
    #         acceptance_ratios
    #     )

    #     # Find first rejection
    #     try:
    #         n_accepted = torch.where(~accepts)[0][0].item()
    #     except:
    #         n_accepted = len(accepts)

    #     return n_accepted

    def verify_tokens(
        self,
        target_probs: torch.Tensor,
        draft_tokens: torch.Tensor,
        draft_probs: torch.Tensor,
        temperature: float = 1.0
    ) -> int:
        """Determine number of accepted tokens"""
        # Get target probabilities for the draft tokens
        target_probs_draft_tokens = target_probs.gather(
            -1,
            draft_tokens.unsqueeze(-1)
        ).squeeze(-1)

        target_probs_draft_tokens = target_probs_draft_tokens / max(temperature, 1e-10)
        draft_probs = draft_probs / max(temperature, 1e-10)

        # Calculate acceptance ratios
        acceptance_ratios = target_probs_draft_tokens.float() / draft_probs

        # Sample uniform random numbers 
        random_nums = torch.zeros_like(target_probs_draft_tokens).float().uniform_()

        mask = random_nums > acceptance_ratios
        num_accepted = (mask.cumsum(dim = -1) == 0).sum(dim = -1)

        return num_accepted.int().item()

    def translate(
        self,
        source_text: str,
        max_length: int = 128
    ) -> str:
        """Translate source text using speculative decoding."""
        # Encode source text
        encoder_inputs = self.tokenizer(
            f"translate English to German: {source_text}",
            return_tensors="pt",
            padding=True
        ).to(self.device)

        # Initialize with start token
        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)

        output = self.target_model(
            input_ids=encoder_inputs.input_ids,
            attention_mask=encoder_inputs.attention_mask,
            decoder_input_ids=decoder_input_ids,
            return_dict=True
        )

        probs = output.logits[:, -1, :]
                    
        probs = F.softmax(probs, dim=-1)
        token_id = torch.multinomial(probs, num_samples=1)

        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id, token_id.item()]], device=self.device)


        total_tokens = 0
        accepted_tokens = 0

        while decoder_input_ids.shape[1] < max_length:
            # Get draft tokens autoregressively
            draft_tokens, draft_probs = self.get_draft_logits(
                encoder_inputs.input_ids,
                encoder_inputs.attention_mask,
                decoder_input_ids,
                self.gamma
            )

            draft_tokens = torch.tensor(draft_tokens, device=self.device)
            draft_probs = torch.tensor(draft_probs, device=self.device)

            if len(draft_tokens) == 0:
                raise ValueError("Draft tokens not generated.")

            # Get target probabilities in parallel
            # start = time.time()
            target_probs, target_logits = self.get_target_probs(
                encoder_inputs.input_ids,
                encoder_inputs.attention_mask,
                decoder_input_ids,
                draft_tokens
            )
            # print("Time taken for target probs: ", time.time() - start)

            # Verify tokens
            n_accepted = self.verify_tokens(target_probs, draft_tokens, draft_probs, self.temperature)
            # print(n_accepted)

            # Accept verified tokens
            if n_accepted > 0:
                decoder_input_ids = torch.cat([
                    decoder_input_ids,
                    draft_tokens[:n_accepted].unsqueeze(0)
                ], dim=1)
                
            with torch.no_grad():
                # n_rejected = self.gamma - n_accepted
                n_rejected = len(draft_tokens) - n_accepted # CHECK IF THIS IS CORRECT
                total_tokens += len(draft_tokens) #self.gamma
                accepted_tokens += n_accepted
                if n_rejected > 0:
                    probs = target_logits[:, -n_rejected, :] #- draft_logits[:, 1-n_rejected, :]
                else:
                    probs = target_logits[:, -1, :]
                    
                probs = F.softmax(probs, dim=-1)
                token_id = torch.multinomial(probs, num_samples=1)

                decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)

            # Check for end of sequence
            if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                break

        # Decode translation
        translation = self.tokenizer.decode(
            decoder_input_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        perc_accepted = accepted_tokens / total_tokens * 100
        return translation, perc_accepted

In [5]:
# speculative_decoder = SpeculativeDecoder(gamma=2)

# source_text = "Obama receives Netanyahu"

# speculative_decoder.translate(source_text)
# start = time.time()
# speculative_translation, pc = speculative_decoder.translate(source_text)
# end = time.time()
# print(f"Speculative translation: {speculative_translation}")
# print(f"Percentage tokens accepted: {pc:.2f}%")
# print(f"Time taken: {end - start:.4f} seconds")

# Normal

In [6]:
class NormalDecoder:
    def __init__(
        self,
        model_name: str = "google-t5/t5-3b",
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
    ):
        self.device = device

        # Initialize tokenizer and model
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        # self.model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto')
        self.model.eval()

    def get_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor
    ) -> torch.Tensor:
        """Get logits from model for the last token."""
        with torch.no_grad():
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                return_dict=True
            )
            return outputs.logits[:, -1, :]

    def sample_token(self, logits: torch.Tensor, temperature: float = 1.0) -> Tuple[torch.Tensor, torch.Tensor]:
        """Sample a token from logits using temperature sampling."""
        if temperature == 0:
            # Greedy sampling
            token_id = torch.argmax(logits, dim=-1)
            prob = torch.ones_like(token_id, dtype=torch.float)
        else:
            # Temperature sampling
            probs = F.softmax(logits / temperature, dim=-1)
            token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
            prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)
        return token_id, prob

    def translate(
        self,
        source_text: str,
        max_length: int = 128,
        temperature: float = 0.7
    ) -> str:
        """Translate source text using the normal T5 model."""
        # Encode source text
        encoder_inputs = self.tokenizer(
            f"translate English to German: {source_text}",
            return_tensors="pt",
            padding=True
        ).to(self.device)

        # Initialize decoder input with start token
        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)

        while decoder_input_ids.shape[1] < max_length:
            # Generate logits for the next token
            logits = self.get_logits(
                encoder_inputs.input_ids,
                encoder_inputs.attention_mask,
                decoder_input_ids
            )

            # Sample a token
            token_id, _ = self.sample_token(logits, temperature)

            # Add token to the decoder input
            decoder_input_ids = torch.cat(
                [decoder_input_ids, token_id.view(1, 1)],
                dim=1
            )

            # Break if end token is generated
            if token_id.item() == self.tokenizer.eos_token_id:
                break

        # Decode and return translation
        translation = self.tokenizer.decode(
            decoder_input_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        return translation


In [7]:
# speculative_decoder = SpeculativeDecoder()
# normal_decoder = NormalDecoder()

# source_text = "He said Lamb made the fateful 911 call sometime after that." # spec does not work

# speculative_decoder.translate(source_text)
# start = time.time()
# speculative_translation, pc = speculative_decoder.translate(source_text)
# end = time.time()
# print(f"Speculative translation: {speculative_translation}")
# print(f"Percentage tokens accepted: {pc:.2f}%")
# print(f"Time taken: {end - start:.4f} seconds")

# start = time.time()
# normal_translation = normal_decoder.translate(source_text)
# end = time.time()
# print(f"Normal translation: {normal_translation}")
# print(f"Time taken: {end - start:.4f} seconds")

In [8]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=4, temperature=0.1)
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['test']['translation'][:30]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    # if normal_time - spec_time > -0.1:
    spec_total_time += spec_time
    normal_total_time += normal_time
    total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
  3%|▎         | 1/30 [00:03<01:33,  3.21s/it]

Source: Obama receives Netanyahu
Normal Translation: Obama empfing Netanjahu
Time taken: 0.72 seconds
Speculative Translation: Obama Netanjahu empfing
Time taken: 2.49 seconds
Percentage tokens accepted: 60.00%
Target: Obama empfängt Netanyahu


  7%|▋         | 2/30 [00:07<01:50,  3.94s/it]

Source: The relationship between Obama and Netanyahu is not exactly friendly.
Normal Translation: Die Beziehung zwischen Obama und Netanjahu ist nicht gerade freundschaftlich.
Time taken: 1.64 seconds
Speculative Translation: Die Beziehungen zwischen Obama und Netanjahu sind nichtakt. Andersgesagt, das  gegenebul Klima gibt es in Europa nie. Nicht: wir diese 1995 erträumtenk Revolution in Berlin
Time taken: 2.81 seconds
Percentage tokens accepted: 42.86%
Target: Das Verhältnis zwischen Obama und Netanyahu ist nicht gerade freundschaftlich.


 10%|█         | 3/30 [00:13<02:11,  4.87s/it]

...

Source: This year, he was only teaching two online classes.
Normal Translation: Dieses Jahr hat er nur noch zwei Online-Klassen gehalten.
Time taken: 1.48 seconds
Speculative Translation: In diesem Jahr gab dieser nur Online in zweienungen.
Time taken: 1.06 seconds
Percentage tokens accepted: 32.00%
Target: In diesem Jahr unterrichtete er nur zwei Online-Kurse.


 97%|█████████▋| 29/30 [03:05<00:05,  5.51s/it]

Source: Recent changes in the university's hiring policies meant that the doctorate Lamb had worked so hard to earn would not guarantee him an automatic tenure track to become an assistant professor.
Normal Translation: Nach der langer, mühsamen Arbeit, die ihn zum Doctorat beworben hatte, war er nicht mehr der ersten Hund, der als Assistent Professor zugelassen war.
Time taken: 4.01 seconds
Speculative Translation: Der Staat hatte sich zum "Spal" für Länder entscheiden die zahlzahlstechnischstengleichen ausgewiesen Dieliche.
Time taken: 1.85 seconds
Percentage tokens accepted: 34.78%
Target: Neueste Änderungen in der Beschäftigungspolitik der Universität bedeuteten, dass die Promotion, für die Lamb so hart gearbeitet hatte, für ihn keine Garantie für einen automatischen Weg zu einer Anstellung als Assistant-Professor darstellen würde.


100%|██████████| 30/30 [03:13<00:00,  6.44s/it]

Source: University President William LaForge said he didn't know of any conflict between Lamb and Schmidt but "obviously there was something in Mr. Lamb's mind."
Normal Translation: University President William LaForge said he didn't know of any conflict between Lamb and Schmidt but "obviously there was something in Mr. Lamb's mind"
Time taken: 3.30 seconds
Speculative Translation: "athes"." Aber  der neue desgroßen Fix sind auch neue Förderungen vons gehen die Char oder Behung Chsn, hergekommen Mar : "ncel...e  mls." (dh
Time taken: 4.57 seconds
Percentage tokens accepted: 18.97%
Target: Universitäts-Präsident William LaForge sagte, er wisse nichts von einem Konflikt zwischen Lamb und Schmidt, aber "natürlich gab es etwas in Mr Lambs Vorstellung."

Average time taken for normal decoding: 3.39 seconds
Average time taken for speculative decoding: 3.05 seconds
Average speedup over 30 iterations: 1.11x


In [ ]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=8, temperature=0.1, draft_model_name='google-t5/t5-base')
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['test']['translation'][:30]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    if normal_time - spec_time > -0.1:
        spec_total_time += spec_time
        normal_total_time += normal_time
        total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

In [ ]:
# Initialize decoder
from tqdm import tqdm

speculative_decoder = SpeculativeDecoder(gamma=4, temperature=0.1, draft_model_name='google-t5/t5-large')
normal_decoder = NormalDecoder()

spec_total_time = 0
normal_total_time = 0
total_iters = 0

for i in tqdm(en_gr_dataset['test']['translation'][:100]):
    source_text = i['en']
    target_text = i['de']
    
    # Time the translation
    start_time = time.time()
    spec_translation, pc = speculative_decoder.translate(source_text)
    end_time = time.time()

    spec_time = end_time - start_time

    # spec_total_time += spec_time
    
    start_time = time.time()
    normal_translation = normal_decoder.translate(source_text)
    end_time = time.time()

    normal_time = end_time - start_time

    # normal_total_time += normal_time

    print(f"Source: {source_text}")
    print(f"Normal Translation: {normal_translation}")
    print(f"Time taken: {normal_time:.2f} seconds")
    print(f"Speculative Translation: {spec_translation}")
    print(f"Time taken: {spec_time:.2f} seconds")
    print(f"Percentage tokens accepted: {pc:.2f}%")
    
    print(f"Target: {target_text}")

    if normal_time - spec_time > -0.1:
        spec_total_time += spec_time
        normal_total_time += normal_time
        total_iters += 1

print(f"\nAverage time taken for normal decoding: {normal_total_time / total_iters:.2f} seconds")
print(f"Average time taken for speculative decoding: {spec_total_time / total_iters:.2f} seconds")
print(f"Average speedup over {total_iters} iterations: {normal_total_time / spec_total_time:.2f}x")

# Online Spec

In [ ]:
class OnlineSpeculativeDecoder:
    def __init__(
        self,
        target_model_name = "google-t5/t5-large",
        draft_model_name = "google-t5/t5-small",
        device = "cuda" if torch.cuda.is_available() else "cpu",
        gamma = 4,
        update_interval=2,  # Update draft model after every `update_interval` iterations
        buffer_size_threshold=2,  # Buffer size threshold for updates
        time_threshold=2,  # Time threshold (in seconds) for updates
    ):
        self.device = device
        self.gamma = gamma
        self.update_interval = update_interval
        self.buffer_size_threshold = buffer_size_threshold
        self.time_threshold = time_threshold
        self.last_update_time = time.time()  # Track last update time

        self.tokenizer = T5Tokenizer.from_pretrained(target_model_name)
        self.target_model = T5ForConditionalGeneration.from_pretrained(target_model_name).to(device)
        self.draft_model = T5ForConditionalGeneration.from_pretrained(draft_model_name).to(device)

        self.target_model.eval()
        self.draft_model.eval()

        # Buffers for storing token proposals and updates
        self.replay_buffer = []
        self.temp_buffer = []  # Temporary buffer for a single request

        # Counter for iteration tracking
        self.iteration_count = 0

    def get_draft_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        gamma: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get draft logits for gamma tokens"""
        draft_tokens = []
        draft_probs = []
        current_decoder_ids = decoder_input_ids

        # Generate gamma tokens from the draft model
        for _ in range(gamma):
            with torch.no_grad():
                outputs = self.draft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=current_decoder_ids,
                    return_dict=True
                )
                logits = outputs.logits[:, -1, :]  # Get logits for last position
                probs = F.softmax(logits, dim=-1)

                # Sample token
                token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
                prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)

                draft_tokens.append(token_id.item())
                draft_probs.append(prob.item())

                # Update decoder inputs for next iteration
                current_decoder_ids = torch.cat(
                    [current_decoder_ids, token_id.view(1, 1)],
                    dim=1
                )

                if token_id.item() == self.tokenizer.eos_token_id:
                    break

        return draft_tokens, draft_probs, current_decoder_ids, outputs.logits

    def get_target_probs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        draft_tokens: torch.Tensor
    ) -> torch.Tensor:
        """Get target probabilities for the draft tokens in parallel."""
        with torch.no_grad():
            # Add draft tokens to decoder input
            full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)

            outputs = self.target_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=full_decoder_ids,
                return_dict=True
            )

            # Get probabilities for positions before each draft token
            logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
            target_probs = F.softmax(logits, dim=-1)

            return target_probs.squeeze(0), outputs.logits
        
    def get_logits(self, model, input_ids, attention_mask):
        return model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        ).logits

    def verify_tokens(
        self,
        target_probs: torch.Tensor,
        draft_tokens: torch.Tensor,
        draft_probs: torch.Tensor,
    ) -> int:
        """Determine number of accepted tokens"""
        # Get target probabilities for the draft tokens
        target_probs_draft_tokens = target_probs.gather(
            -1,
            draft_tokens.unsqueeze(-1)
        ).squeeze(-1)

        # Calculate acceptance ratios
        acceptance_ratios = target_probs_draft_tokens / draft_probs

        # Sample uniform random numbers
        random_nums = torch.rand_like(acceptance_ratios)

        # Find number of accepted tokens
        # Accept if random number < min(1, target_prob / draft_prob)
        accepts = random_nums < torch.minimum(
            torch.ones_like(acceptance_ratios),
            acceptance_ratios
        )

        # Find first rejection
        try:
            n_accepted = torch.where(~accepts)[0][0].item()
        except:
            n_accepted = len(accepts)

        return n_accepted

        # accepted_tokens = []
        # for i in range(len(draft_tokens)):
        #     if target_probs[i] / draft_probs[i] > torch.rand(1).item():
        #         accepted_tokens.append(draft_tokens[i])
        #     else:
        #         break # Stop if token is not accepted

        # return len(accepted_tokens)
    
    # TODO: verify this, might need to do some window size thing
    def update_draft_model(self):
        """Update draft model with the replay buffer."""
        if len(self.replay_buffer) == 0:
            return

        # Get draft tokens, draft and target probabilities from the replay buffer
        # draft_tokens = torch.tensor([x[0] for x in self.replay_buffer], device=self.device)
        # print(self.replay_buffer[0][0])
        # draft_probs = self.replay_buffer[:, 0]
        # target_probs = self.replay_buffer[:, 1]
        draft_probs = torch.stack([x[0][0] for x in self.replay_buffer], dim=0)
        target_probs = torch.stack([x[1][0] for x in self.replay_buffer], dim=0)

        self.draft_model.train()

        # for param in self.draft_model.parameters():
        #     print(param.requires_grad)


        # criterion = torch.nn.CrossEntropyLoss()
        # print(draft_probs.shape, target_probs.shape)
        loss = self.soft_cross_entropy(draft_probs, target_probs)
        print("Loss grad_fn:", loss.grad_fn)
        print("Draft probs grad_fn:", draft_probs.grad_fn)
        print("Target probs grad_fn:", target_probs.grad_fn)

        loss.backward()

        self.draft_model.eval()

        # Clear the replay buffer
        self.replay_buffer = []

    def soft_cross_entropy(self, predicts, targets, padding_mask=None):
        predict_log_prob = torch.nn.functional.log_softmax(predicts, dim=-1)
        targets_prob = torch.nn.functional.softmax(targets, dim=-1)
        entropy = -targets_prob * predict_log_prob
        # expand_mask = padding_mask.unsqueeze(-1).expand_as(entropy)
        # entropy.masked_fill_(expand_mask, 0)
        # mean_entropy = entropy.sum() / (~padding_mask).sum()
        return entropy

    def translate(
        self,
        source_text: str,
        max_length: int = 128
    ) -> str:
        """Translate source text using speculative decoding."""
        # Encode source text
        encoder_inputs = self.tokenizer(
            f"translate English to German: {source_text}",
            return_tensors="pt",
            padding=True
        ).to(self.device)

        # Initialize with start token
        decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)

        self.iteration_count = 0
        self.replay_buffer = []

        while decoder_input_ids.shape[1] < max_length:
            self.temp_buffer = []

            while decoder_input_ids.shape[1] < max_length:
                # Get draft tokens autoregressively
                # print("Encoder Inputs", encoder_inputs.input_ids.shape)
                draft_tokens, draft_probs, proposed_decoder_ids, draft_logits = self.get_draft_logits(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    self.gamma
                )

                draft_tokens = torch.tensor(draft_tokens, device=self.device)
                draft_probs = torch.tensor(draft_probs, device=self.device)

                if len(draft_tokens) == 0:
                    break

                # Get target probabilities in parallel
                target_probs, target_logits = self.get_target_probs(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    draft_tokens
                )
                # print(draft_logits.shape, target_logits.shape)

                # Verify tokens
                n_accepted = self.verify_tokens(target_probs, draft_tokens, draft_probs)

                # Accept verified tokens
                if n_accepted > 0:
                    decoder_input_ids = torch.cat([
                        decoder_input_ids,
                        draft_tokens[:n_accepted].unsqueeze(0)
                    ], dim=1)

                # # If rejection or no acceptance, sample one token from target
                # if n_accepted < len(draft_tokens):
                #     with torch.no_grad():
                #         outputs = self.target_model(
                #             input_ids=encoder_inputs.input_ids,
                #             attention_mask=encoder_inputs.attention_mask,
                #             decoder_input_ids=decoder_input_ids,
                #             return_dict=True
                #         )
                #         logits = outputs.logits[:, -1, :]
                #         probs = F.softmax(logits, dim=-1)
                #         token_id = torch.multinomial(probs, num_samples=1)
                #         decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)

                with torch.no_grad():
                    # print(target_logits.shape, draft_logits.shape)
                    probs = target_logits[:, -1, :] #- draft_logits[:, -1, :]
                    probs = F.softmax(probs, dim=-1)
                    token_id = torch.multinomial(probs, num_samples=1)
                    # print(probs.shape, token_id.shape)
                    decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)

                # Check for end of sequence
                if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                    break
                
                # TODO: Update buffer with draft and target logits of the first rejected token, verify implementation
                # rejected_tokens = draft_tokens[n_accepted]
                if n_accepted < len(draft_tokens):
                    # rejected_prob_draft = draft_logits[:, n_accepted, :]
                    # rejected_prob_target = target_logits[:, n_accepted, :]

                    self.temp_buffer.append((draft_logits[:, -1, :], target_logits[:, -1, :]))

                # Check for end of sequence
                if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                    break

            self.replay_buffer.extend(self.temp_buffer)
            self.iteration_count += 1

            if self.iteration_count % self.update_interval == 0:
                self.update_draft_model()
                self.iteration_count = 0

        # Decode translation
        translation = self.tokenizer.decode(
            decoder_input_ids[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )
        return translation

In [ ]:
# Initialize decoder
online_decoder = OnlineSpeculativeDecoder()

# Example translation
source_text = "In a world where technology evolves at an unprecedented pace, individuals and organizations must adapt quickly to the rapid advancements in artificial intelligence, machine learning, and automation, ensuring that ethical considerations, environmental sustainability, and equitable access to resources are prioritized to create a future that benefits all of humanity."

# Time the translation
start_time = time.time()
translation = online_decoder.translate(source_text)
end_time = time.time()

print(f"Source: {source_text}")
print(f"Translation: {translation}\n")
print(f"Time taken: {end_time - start_time:.2f} seconds")

# Online V2

In [ ]:
class OnlineSpeculativeDecoder:
    def __init__(
        self,
        target_model_name = "google-t5/t5-large",
        draft_model_name = "google-t5/t5-small",
        device = "cuda" if torch.cuda.is_available() else "cpu",
        gamma = 4,
        update_interval=2,  # Update draft model after every `update_interval` iterations
        buffer_size_threshold=2,  # Buffer size threshold for updates
        time_threshold=2,  # Time threshold (in seconds) for updates
    ):
        self.device = device
        self.gamma = gamma
        self.update_interval = update_interval
        self.buffer_size_threshold = buffer_size_threshold
        self.time_threshold = time_threshold
        self.last_update_time = time.time()  # Track last update time

        self.tokenizer = T5Tokenizer.from_pretrained(target_model_name)
        self.target_model = T5ForConditionalGeneration.from_pretrained(target_model_name).to(device)
        self.draft_model = T5ForConditionalGeneration.from_pretrained(draft_model_name).to(device)

        self.target_model.eval()
        self.draft_model.eval()

        # Buffers for storing token proposals and updates
        self.replay_buffer = []
        self.temp_buffer = []  # Temporary buffer for a single request

        # Counter for iteration tracking
        self.iteration_count = 0

    def get_draft_logits(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        gamma: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get draft logits for gamma tokens"""
        draft_tokens = []
        draft_probs = []
        current_decoder_ids = decoder_input_ids

        # Generate gamma tokens from the draft model
        for _ in range(gamma):
            with torch.no_grad():
                outputs = self.draft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_input_ids=current_decoder_ids,
                    return_dict=True
                )
                logits = outputs.logits[:, -1, :]  # Get logits for last position
                probs = F.softmax(logits, dim=-1)

                # Sample token
                token_id = torch.multinomial(probs, num_samples=1).squeeze(-1)
                prob = probs.gather(-1, token_id.unsqueeze(-1)).squeeze(-1)

                draft_tokens.append(token_id.item())
                draft_probs.append(prob.item())

                # Update decoder inputs for next iteration
                current_decoder_ids = torch.cat(
                    [current_decoder_ids, token_id.view(1, 1)],
                    dim=1
                )

                if token_id.item() == self.tokenizer.eos_token_id:
                    break

        return draft_tokens, draft_probs, current_decoder_ids, outputs.logits

    def get_target_probs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        decoder_input_ids: torch.Tensor,
        draft_tokens: torch.Tensor
    ) -> torch.Tensor:
        """Get target probabilities for the draft tokens in parallel."""
        with torch.no_grad():
            # Add draft tokens to decoder input
            full_decoder_ids = torch.cat([decoder_input_ids, draft_tokens.unsqueeze(0)], dim=1)

            outputs = self.target_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=full_decoder_ids,
                return_dict=True
            )

            # Get probabilities for positions before each draft token
            logits = outputs.logits[:, -(len(draft_tokens) + 1):-1, :]
            target_probs = F.softmax(logits, dim=-1)

            return target_probs.squeeze(0), outputs.logits
        
    def get_logits(self, model, input_ids, attention_mask, decoder_input_ids):
        return model(
            input_ids=input_ids,
            decoder_input_ids=decoder_input_ids,
            attention_mask=attention_mask,
        ).logits

    def verify_tokens(
        self,
        target_probs: torch.Tensor,
        draft_tokens: torch.Tensor,
        draft_probs: torch.Tensor,
    ) -> int:
        """Determine number of accepted tokens"""
        # Get target probabilities for the draft tokens
        target_probs_draft_tokens = target_probs.gather(
            -1,
            draft_tokens.unsqueeze(-1)
        ).squeeze(-1)

        # Calculate acceptance ratios
        acceptance_ratios = target_probs_draft_tokens / draft_probs

        # Sample uniform random numbers
        random_nums = torch.rand_like(acceptance_ratios)

        # Find number of accepted tokens
        # Accept if random number < min(1, target_prob / draft_prob)
        accepts = random_nums < torch.minimum(
            torch.ones_like(acceptance_ratios),
            acceptance_ratios
        )

        # Find first rejection
        try:
            n_accepted = torch.where(~accepts)[0][0].item()
        except:
            n_accepted = len(accepts)

        return n_accepted

        # accepted_tokens = []
        # for i in range(len(draft_tokens)):
        #     if target_probs[i] / draft_probs[i] > torch.rand(1).item():
        #         accepted_tokens.append(draft_tokens[i])
        #     else:
        #         break # Stop if token is not accepted

        # return len(accepted_tokens)
    
    # TODO: verify this, might need to do some window size thing
    # def update_draft_model(self):
    #     """Update draft model with the replay buffer."""
    #     if len(self.replay_buffer) == 0:
    #         return

    #     # Get draft tokens, draft and target probabilities from the replay buffer
    #     draft_tokens = torch.tensor([x[0] for x in self.replay_buffer], device=self.device)
    #     target_logits = torch.tensor([x[1] for x in self.replay_buffer], device=self.device)

    #     encoder_inputs = s
    #     output = self.draft_model(
    #         input_ids=encoder_inputs.input_ids,
    #         attention_mask=encoder_inputs.attention_mask,
    #         decoder_input_ids=decoder_input_ids,
    #         return_dict=True
    #     )

    def pad_to_2d(self, tensor_list, pad_token_id, max_len=None):
        if not isinstance(tensor_list[0], torch.Tensor):
            tensor_list = [torch.tensor(t).reshape(1, -1) for t in tensor_list]
        if max_len is None:
            max_len = max([t.shape[-1] for t in tensor_list])
        assert max_len > 0

        # Pad each tensor to the max length and stack them to form a 2D tensor
        result = torch.cat(
            [
                torch.nn.functional.pad(
                    tensor, (0, max_len - tensor.shape[-1]),
                    value=pad_token_id
                )
                for tensor in tensor_list
            ],
            dim=0
        )
        return result
        

    def soft_cross_entropy(self, predicts, targets, padding_mask=None):
        predict_log_prob = torch.nn.functional.log_softmax(predicts, dim=-1)
        targets_prob = torch.nn.functional.softmax(targets, dim=-1)
        entropy = -targets_prob * predict_log_prob
        expand_mask = padding_mask.unsqueeze(-1).expand_as(entropy)
        entropy.masked_fill_(expand_mask, 0)
        mean_entropy = entropy.sum() / (~padding_mask).sum()
        return mean_entropy

    def translate_dataset(
        self,
        sentences: List[str],
        max_length: int = 128
    ) -> List[str]:
        """Translate dataset using online speculative decoding."""

        self.iteration_count = 0
        self.replay_buffer = []

        translated_data = []

        for source_text in sentences:
            # Encode source text
            encoder_inputs = self.tokenizer(
                f"translate English to German: {source_text}",
                return_tensors="pt",
                padding=True
            ).to(self.device)

            # Initialize with start token
            decoder_input_ids = torch.tensor([[self.tokenizer.pad_token_id]], device=self.device)
            self.temp_buffer = []

            while decoder_input_ids.shape[1] < max_length:

                # Check for end of sequence
                if decoder_input_ids[0][-1].item() == self.tokenizer.eos_token_id:
                    break

                # Get draft tokens autoregressively
                # print("Encoder Inputs", encoder_inputs.input_ids.shape)
                draft_tokens, draft_probs, proposed_decoder_ids, draft_logits = self.get_draft_logits(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    self.gamma
                )

                draft_tokens = torch.tensor(draft_tokens, device=self.device)
                draft_probs = torch.tensor(draft_probs, device=self.device)

                if len(draft_tokens) == 0:
                    break

                # Get target probabilities in parallel
                target_probs, target_logits = self.get_target_probs(
                    encoder_inputs.input_ids,
                    encoder_inputs.attention_mask,
                    decoder_input_ids,
                    draft_tokens
                )
                # print(draft_logits.shape, target_logits.shape)

                # Verify tokens
                n_accepted = self.verify_tokens(target_probs, draft_tokens, draft_probs)

                # Accept verified tokens
                if n_accepted > 0:
                    decoder_input_ids = torch.cat([
                        decoder_input_ids,
                        draft_tokens[:n_accepted].unsqueeze(0)
                    ], dim=1)

                with torch.no_grad():
                    # print(target_logits.shape, draft_logits.shape)
                    probs = target_logits[:, -1, :] #- draft_logits[:, -1, :]
                    probs = F.softmax(probs, dim=-1)
                    token_id = torch.multinomial(probs, num_samples=1)
                    # print(probs.shape, token_id.shape)
                    decoder_input_ids = torch.cat([decoder_input_ids, token_id], dim=1)
                
                # rejected_tokens = draft_tokens[n_accepted]
                if n_accepted < len(draft_tokens):

                    self.temp_buffer.append((encoder_inputs, decoder_input_ids, target_logits, n_accepted))

            self.replay_buffer.extend(self.temp_buffer)
            self.iteration_count += 1

            if self.iteration_count % self.update_interval == 0:
                # self.update_draft_model()
                self.draft_model.train()
                
                # finetune over collected tokens and logits
                encoder_input_ids = self.pad_to_2d([x[0].input_ids for x in self.replay_buffer], 0)
                encoder_attention_mask = torch.stack([x[0].attention_mask[0] for x in self.replay_buffer], dim=0)
                decoder_input_ids = self.pad_to_2d([x[1] for x in self.replay_buffer], 0, 512)

                print(encoder_input_ids.shape, encoder_attention_mask.shape, decoder_input_ids.shape)

                target_logits = [x[2] for x in self.replay_buffer]
                for i in range(len(target_logits)):
                    temp = torch.zeros(1, 32128, device=self.device).repeat(512 - target_logits[i].shape[1], 1).unsqueeze(0)
                    target_logits[i] = torch.cat([target_logits[i], temp], dim=1)

                n_accepted_tokens = [x[3] for x in self.replay_buffer]

                # CUDA out of memory error
                draft_logits = self.get_logits(self.draft_model, encoder_input_ids, encoder_attention_mask, decoder_input_ids).float()

                # need to get loss only using the wrong tokens
                # TODO: check if we need to ignore the pad tokens in the mask
                mask = torch.ones_like(decoder_input_ids, dtype=torch.bool)
                for i in range(len(n_accepted_tokens)):
                    mask[i, n_accepted_tokens[i]:] = False
                loss = self.soft_cross_entropy(draft_logits, target_logits, mask)
                loss.backward()

                self.draft_model.eval()
                self.replay_buffer = []
                self.iteration_count = 0

            # Decode translation
            translation = self.tokenizer.decode(
                decoder_input_ids[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            translated_data.append(translation)
        return translated_data

In [ ]:
# Initialize decoder
online_decoder = OnlineSpeculativeDecoder()

# Example translation
source_text = "In a world where technology evolves at an unprecedented pace, individuals and organizations must adapt quickly to the rapid advancements in artificial intelligence, machine learning, and automation, ensuring that ethical considerations, environmental sustainability, and equitable access to resources are prioritized to create a future that benefits all of humanity."
sents = [source_text] * 10
# Time the translation
start_time = time.time()
translation = online_decoder.translate_dataset(sents)
end_time = time.time()

for i, sent in enumerate(sents):
    print(f"Source: {sent}")
    print(f"Translation: {translation[i]}\n")
print(f"Time taken: {end_time - start_time:.2f} seconds")